In [8]:
!pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-ia2tgvdl
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-ia2tgvdl
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [2]:
from avalanche.benchmarks.classic import SplitMNIST
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.models import SimpleMLP
from avalanche.training.strategies import Naive, CWRStar, Replay, GDumb, Cumulative, LwF, GEM, AGEM, EWC
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.training.plugins import EvaluationPlugin

In [5]:
scenario = SplitMNIST(n_experiences=5)
model = SimpleMLP(num_classes=scenario.n_classes)

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=10, save_image=False, stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True))

/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:84: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  "No benchmark provided to the evaluation plugin. "
/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:93: UserWarning: No loggers specified, metrics will not be logged
  warnings.warn('No loggers specified, metrics will not be logged')


In [6]:
# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)

In [9]:
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 5]
Training completed
Computing accuracy on the whole test set
Start of experience:  1
Current Classes:  [9, 7]
Training completed
Computing accuracy on the whole test set
Start of experience:  2
Current Classes:  [2, 4]
Training completed
Computing accuracy on the whole test set
Start of experience:  3
Current Classes:  [1, 3]
Training completed
Computing accuracy on the whole test set
Start of experience:  4
Current Classes:  [8, 6]
Training completed
Computing accuracy on the whole test set


In [10]:
results

[{'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000': 100.21760180705009,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001': 98.89388206488088,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp002': 99.84360099531469,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp003': 100.55796344531878,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp004': 99.78072231282418,
  'ConfusionMatrix_Stream/eval_phase/test_stream': tensor([[ 950,    0,    0,    0,    0,   30,    0,    0,    0,    0],
          [   0,    0,    0,    0,    0, 1135,    0,    0,    0,    0],
          [ 310,    0,    0,    0,    0,  722,    0,    0,    0,    0],
          [  97,    0,    0,    0,    0,  913,    0,    0,    0,    0],
          [ 136,    0,    0,    0,    0,  846,    0,    0,    0,    0],
          [  67,    0,    0,    0,    0,  825,    0,    0,    0,    0],
          [ 378,    0,    0,    0,    0,  580,    0,    0,    0,    0],
          [ 102,    0,    0,    0,    0,  926,    0,    0,    0,    0]